In [1]:
from PIL import Image
import numpy as np
import glob
import os
from configs.scannet_constants import *
from tqdm.notebook import tqdm
import argparse
import pandas as pd
import math

In [43]:
THRESHOLD = 30 ## Number of pixels to start from the beginning and end from the end of the image to crete the center.

gt_dir = "/home/rozenberszki/Downloads/ScanNet-gt-124/"
list_dirs = sorted(glob.glob(gt_dir+'/**', recursive=False))
class_dist = {}
class_dist[0] = {
    "sum":0,
    "scenes":{

    }
}
# for i in range(21):
#     if(i==0): continue
#     class_dist[VALID_CLASS_IDS_20[i-1]]=0
for key in VALID_CLASS_IDS_20:
    class_dist[key]={
        "sum":0,
        "scenes":{

        }
    }

scene_def = {
    "poses":[

    ]
}

In [44]:
for tqdm_idx in tqdm(range(len(list_dirs))):
    scene = list_dirs[tqdm_idx]
    if(scene.endswith("_01")): continue
    if(scene.endswith("_02")): continue
    for image in sorted(glob.glob(os.path.join(gt_dir, scene, '**'), recursive=False)):
        if(image.endswith(".png")): continue
        img = np.load(image, allow_pickle=True)
        scene_id = scene.split("/")[-1]
        pose_id = image.split("/")[-1]
        shrink = img
        shrink = shrink[THRESHOLD:shrink.shape[0]-THRESHOLD,THRESHOLD:shrink.shape[1]-THRESHOLD]
        img = shrink
        for key in range(21):
            if(key==0): 
                sum_pts = (img==key).sum()
                class_dist[key]["sum"]+=(img==key).sum()
                if(sum_pts > 0):
                    if(scene_id not in class_dist[key]["scenes"]):
                        class_dist[key]["scenes"][scene_id]={
                        "sum":(img==key).sum(),
                        "poses":{
                            pose_id: (img==key).sum()
                        }  
                        }
                    else:
                        class_dist[key]["scenes"][scene_id]["sum"]+=(img==key).sum()
                        class_dist[key]["scenes"][scene_id]["poses"][pose_id] = (img==key).sum()
                continue
            new_key = VALID_CLASS_IDS_20[key-1] ## Key
            # idx = img==key
            sum_pts = (img==key).sum()
            class_dist[new_key]["sum"]+= sum_pts
            if(sum_pts > 0):
                if(scene_id not in class_dist[new_key]["scenes"]):
                    class_dist[new_key]["scenes"][scene_id]={
                        "sum":(img==key).sum(),
                        "poses":{
                        pose_id: (img==key).sum()
                        }  
                    }
                else:
                    class_dist[new_key]["scenes"][scene_id]["sum"]+=(img==key).sum()
                    class_dist[new_key]["scenes"][scene_id]["poses"][pose_id] = (img==key).sum()

  0%|          | 0/1513 [00:00<?, ?it/s]

In [45]:
np.save("../../Downloads/class_dist_center.npy", class_dist)

## Load the class distribution and Analyze

In [46]:
load = np.load("../../Downloads/class_dist_center.npy", allow_pickle='TRUE').item()

In [47]:
class_ids = list(load.keys())
class_vals = []
for key in load.keys():
    class_vals.append(load[key]["sum"])

In [48]:
classes = list(tuple('i')+CLASS_LABELS_20)
classes[0]="Undefined"
quick_sum = {}
for i, id in enumerate(class_ids):
    quick_sum[id]=class_vals[i]
df = pd.DataFrame(quick_sum.items(), columns=["Class","Values"])
df.loc['Total'] = pd.Series(df['Values'].sum(), index=['Values'])
df['Percentage'] = df['Values']/df['Values'].sum() *200
df['Class_Name'] = classes+["Name"]
sorted_list = df.sort_values(by='Values', ascending=False)
display(sorted_list)

,Class,Values,Percentage,Class_Name
Total,NaN,246714368.0,100.000000,Name
0,0.0,50989862.0,20.667569,Undefined
1,1.0,46644632.0,18.906330,wall
2,2.0,42025711.0,17.034156,floor
5,5.0,19310514.0,7.827073,chair
7,7.0,14879086.0,6.030896,table
3,3.0,11531862.0,4.674175,cabinet
20,39.0,9714106.0,3.937390,otherfurniture
4,4.0,9476360.0,3.841025,bed
8,8.0,9138117.0,3.703926,door


## Define the dome functionality

In [ ]:
def rotate_point_matrix(init_pose, angle, center_point):
    # Translate point to origin
    point = center_point.copy()
    point[1] -= 3
    translate_matrix = np.eye(4)
    translate_matrix[:3,3:] = -point

    # Rotate point
    angle_rad = math.radians(angle)
    rotate_matrix = [
        [math.cos(angle_rad), -math.sin(angle_rad), 0, 0],
        [math.sin(angle_rad), math.cos(angle_rad), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ]
    
    translate_back_matrix = np.eye(4)
    translate_back_matrix[:3, 3:] = point
    transformation_matrix = np.zeros(4)

    transformation_matrix = translate_back_matrix @ rotate_matrix
    transformation_matrix = transformation_matrix @ translate_matrix
    
    return transformation_matrix @ init_pose

In [ ]:
ANGLE = 90
STEP_SIZE = 15

def create_dome(init_pose, center_point):
    for i in range(STEP_SIZE, ANGLE, STEP_SIZE):
        new_pose = rotate_point_matrix(init_pose.copy(), i, center_point=center_point.copy())

## Get the 4 least represented classes

In [49]:
least_4 = sorted_list.iloc[-4:]
display(least_4)

for Class in least_4['Class']:
    print(Class)
    class_key = int(Class)
    print(load[class_key]['sum']) 
    print("Scenes List:")
    for k in load[class_key]['scenes'].keys():
        print(k,":",load[class_key]['scenes'][k]['sum'])

,Class,Values,Percentage,Class_Name
19,36.0,1388134.0,0.562648,bathtub
18,34.0,1320097.0,0.535071,sink
11,11.0,1027260.0,0.416376,picture
16,28.0,1016278.0,0.411925,shower curtain


36.0
1388134
Scenes List:
plenoxel_scannet_scene0029_00 : 49329
plenoxel_scannet_scene0034_00 : 16155
plenoxel_scannet_scene0046_00 : 11249
plenoxel_scannet_scene0054_00 : 16786
plenoxel_scannet_scene0065_00 : 20189
plenoxel_scannet_scene0083_00 : 12980
plenoxel_scannet_scene0100_00 : 1855
plenoxel_scannet_scene0101_00 : 7742
plenoxel_scannet_scene0105_00 : 54763
plenoxel_scannet_scene0116_00 : 31807
plenoxel_scannet_scene0119_00 : 8545
plenoxel_scannet_scene0153_00 : 10006
plenoxel_scannet_scene0170_00 : 12461
plenoxel_scannet_scene0195_00 : 86747
plenoxel_scannet_scene0207_00 : 9069
plenoxel_scannet_scene0221_00 : 366
plenoxel_scannet_scene0237_00 : 15001
plenoxel_scannet_scene0250_00 : 43556
plenoxel_scannet_scene0260_00 : 21846
plenoxel_scannet_scene0265_00 : 30340
plenoxel_scannet_scene0268_00 : 11533
plenoxel_scannet_scene0276_00 : 16361
plenoxel_scannet_scene0346_00 : 4961
plenoxel_scannet_scene0392_00 : 924
plenoxel_scannet_scene0396_00 : 30046
plenoxel_scannet_scene0406_00 : 2

In [3]:
depth = np.load("../../Downloads/ScanNet-gt-124-depth/plenoxel_scannet_scene0000_00/0_depth.npy", allow_pickle='TRUE')

In [6]:
d_sq = depth.squeeze()
print(d_sq)

[[2.293 2.299 2.307 ... 1.553 1.559 1.562]
 [2.297 2.303 2.31  ... 1.555 1.56  1.564]
 [2.3   2.307 2.312 ... 1.558 1.562 1.566]
 ...
 [1.714 1.714 1.714 ... 1.715 1.715 1.715]
 [1.7   1.7   1.7   ... 1.701 1.701 1.702]
 [1.687 1.687 1.687 ... 1.688 1.688 1.688]]
